## Stochastic Gradient Descent

We calculate the loss and gradient based on only **one sample**.   (Not recommended due to unstability)

$$\frac{\partial J}{\partial \theta_j} = (h^{(i)}-y^{(i)})x_j$$

## Mini-Batch Gradient Descent

We calculate the loss and gradient based on **subset of samples**.  (Recommended; used as standard in deep learning)

$$\frac{\partial J}{\partial \theta_j} = \sum_{i=start}^{batch}(h^{(i)}-y^{(i)})x_j$$

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from time import time

diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target
m = X.shape[0]  #number of samples
n = X.shape[1]  #number of features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# actually you can do like this too
# X = np.insert(X, 0, 1, axis=1)
intercept = np.ones((X_train.shape[0], 1))
X_train   = np.concatenate((intercept, X_train), axis=1)
intercept = np.ones((X_test.shape[0], 1))
X_test    = np.concatenate((intercept, X_test), axis=1)

Here, I want to demonstrate some techniques:
- **Class** - it's much better to write any in the class form, as you can modularize it for future use, so please be comfortable with it
- **Early stopping** - it's expensive to run the model until a certain set of iterations; instead, we can stop when the training loss is no longer decreasing
- **Cross validation** - we never really do cross validation from scratch, so here I showed you how to do it

Some terms worth mentioning:
- **Epoch** - it's a popular term in ML/DL - one epoch refers to a training and validation process with all the training data with one model.

Some coding practice worth mentioning:
- Notice how I put `_` in front of some function; it does not really have any functionality aside from telling the coders that it is not meant for outside use (should not be called from `__main__`)

In [2]:
from sklearn.model_selection import KFold

class LinearRegression(object):
    
    #in this class, we add cross validation as well for some spicy code....
    kfold = KFold(n_splits=5)
            
    def __init__(self, alpha=0.001, num_epochs=5, batch_size=50, method='batch', cv=kfold):
        self.alpha      = alpha
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.method     = method
        self.cv         = cv
    
    def mse(self, ytrue, ypred):
        return ((ypred - ytrue) ** 2).sum() / ytrue.shape[0]
    
    def fit(self, X_train, y_train):
        
        #create a list of kfold scores
        self.kfold_scores = list()
        
        #reset val loss
        self.val_loss_old = np.infty

        #kfold.split in the sklearn.....
        #5 splits
        for fold, (train_idx, val_idx) in enumerate(self.cv.split(X_train)):
            
            X_cross_train = X_train[train_idx]
            y_cross_train = y_train[train_idx]
            X_cross_val   = X_train[val_idx]
            y_cross_val   = y_train[val_idx]
            
            #create self.theta here
            self.theta = np.zeros(X_cross_train.shape[1])
            
            #define X_cross_train as only a subset of the data
            #how big is this subset?  => mini-batch size ==> 50
            
            #one epoch will exhaust the WHOLE training set
            for epoch in range(self.num_epochs):
            
                #with replacement or no replacement
                #with replacement means just randomize
                #with no replacement means 0:50, 51:100, 101:150, ......300:323
                #shuffle your index
                perm = np.random.permutation(X_cross_train.shape[0])
                        
                X_cross_train = X_cross_train[perm]
                y_cross_train = y_cross_train[perm]
                
                if   self.method == 'sto':
                    for batch_idx in range(X_cross_train.shape[0]):
                        X_method_train = X_cross_train[batch_idx].reshape(1, -1) #(11,) ==> (1, 11) ==> (m, n)
                        y_method_train = y_cross_train[batch_idx]                    
                        self._train(X_method_train, y_method_train)
                elif self.method == 'mini':
                    for batch_idx in range(0, X_cross_train.shape[0], self.batch_size):
                        #batch_idx = 0, 50, 100, 150
                        X_method_train = X_cross_train[batch_idx:batch_idx+self.batch_size, :]
                        y_method_train = y_cross_train[batch_idx:batch_idx+self.batch_size]
                        self._train(X_method_train, y_method_train)
                else:
                    X_method_train = X_cross_train
                    y_method_train = y_cross_train
                    self._train(X_method_train, y_method_train)
                    
            yhat_val = self.predict(X_cross_val)
            
            #early stopping
            val_loss_new = self.mse(y_cross_val, yhat_val)
            if np.allclose(val_loss_new, self.val_loss_old):
                break
            self.val_loss_old = val_loss_new
            
            self.kfold_scores.append(val_loss_new)
            print(f"Fold {fold}: {val_loss_new}")
                    
    def _train(self, X, y):
        yhat = self.predict(X)
        grad = X.T @(yhat - y)
        self.theta = self.theta - self.alpha * grad
    
    def predict(self, X):
        return X @ self.theta  #===>(m, n) @ (n, )
    
    def _coef(self):
        return self.theta[1:]  #remind that theta is (w0, w1, w2, w3, w4.....wn)
                               #w0 is the bias or the intercept
                               #w1....wn are the weights / coefficients / theta
    def _bias(self):
        return self.theta[0]

In [3]:
lr = LinearRegression(method="batch") #<==try put method="batch" or "sto"
lr.fit(X_train, y_train)
yhat = lr.predict(X_test)
mse  = lr.mse(yhat, y_test)

# print the mse
print("Test MSE: ", mse)

Fold 0: 4995.642541794574
Fold 1: 4035.270261440947
Fold 2: 5465.4331522548455
Fold 3: 5116.4382315849225
Fold 4: 3580.4397606397456
Test MSE:  3789.9012437030083


## Group Workshop - Check your understandings

Answer the following questions:

Instruction:  Gather in your group.  Will randomly pick groups to present.

1.  Using the table of egg price, gold price, etc., please do the followings:
    1.  Draw the vector $\mathbf{x}_2$
    2.  Draw the vector $\mathbf{x}^{(2)}$
    3.  What is $x^{(2)}_1$
    4.  What is $y^{(1)}$
    5.  Draw the vector $\mathbf{y}$
    6.  Draw the entire matrix $\mathbf{X}$
    7.  Create the vector $\mathbf{x}_2$ using numpy
    8.  Create the matrix $\mathbf{X}$ using numpy
2.  Given the first sample $\mathbf{x}^{(1)}$, and given a weight $\boldsymbol{\theta}$ of [2, 0.1, 3], what is $\hat{y}$? Show the calculation by hand.  (this is called *vectorized* form)
3.  Perform 2 using numpy
4.  Given the entire matrix $\mathbf{X}$, and using the same weight as above, what is $\hat{\mathbf{y}}$?  Show the calculation by hand.  This is called *matrix* form.
5.  Perform 4 using numpy
6.  Given the entire matrix $\mathbf{X}$, given a weight $\boldsymbol{\theta}$ of [2, 0.1, 3], and given the $\mathbf{y}$, what is $J(\theta)$? Show the calculation by hand.
7.  Perform 6 using numpy
8.  For training sample 1, i.e., $\mathbf{x}^{(1)}$, what is $\displaystyle \frac{\partial J}{\partial \theta_2}$  Show the calculation by hand.
9.  For all samples, i.e., $\mathbf{X}$, what is $\displaystyle \frac{\partial J}{\partial \theta_2}$  Show the calculation by hand.
10. For all samples, what is $\displaystyle \frac{\partial J}{\partial \theta}$.  Show the calculation by hand.
11. Given $\alpha = 0.01$, what is the new $\boldsymbol{\theta}$?
12. In the beginning of gradient descent, what is the initial weight?
13. Review how to find derivatives/gradients, especially multiplication and chain rule by yourself.
14. Explain in your own words, why finding the derivatives can help you find the better weight.
15. Explain what is stochastic and mini-batch gradient descent.  (actually what you did in 8 is stochastic)
16. Now you have transcended.  Go to the code and understand everything :-)
